In [1]:
import ipynb.fs.full.times_reader as feedreader
import ipynb.fs.full.openai_models as openai
import ipynb.fs.full.gemini_models as gemini
import ipynb.fs.full.data_operations as data
import pandas
import pprint
import json

In [2]:
def main():
    cities = ['Mumbai', 'Delhi', 'Bangalore']
    for city in cities:
        try:
            feedreader.toi_feed_reader.load_links()
            
            print(f'Trying feed for city: {city}...')
            feed = feedreader.toi_feed_reader(city)
            entries = feed.get_feed_entries()
            print(f'Found {entries.shape[0]} news articles for {city}. Now processing to analyse...')
            
            llm = gemini.Gemini_Models()
            response = llm.classify_headlines(entries[['sub-site', 'title']], silent_mode=False)
            if(response.strip()=='' or response.strip()=='{}' or response.strip()=='[]'):
                continue
            else:
                classified_hl = pandas.DataFrame(json.loads(response), columns=['title', 'classification', 'explanation'])
                
                itemized_hl = entries.join(classified_hl, lsuffix='_orig', rsuffix='_copy') 
                itemized_hl = itemized_hl[itemized_hl['classification'] == True]
                itemized_hl.drop_duplicates(subset=['link_id'], keep='first', inplace=True, ignore_index=False)
                
                writer = data.gspread_Operator()
                writer.write_headlines(itemized_hl)
            
        except Exception as e:
            print(f'Caught exception {e} in code flow...')
            import traceback
            traceback.print_exc()
        finally:
            continue

In [3]:
if __name__ == '__main__':
    hl = main()

Trying feed for city: Mumbai...
Found 20 news articles for Mumbai. Now processing to analyse...
Sending titles for classification to Gemini model...
Input is of length: 20.
Wrote 7 lines to Links sheet in the workbook
Trying feed for city: Delhi...
Found 20 news articles for Delhi. Now processing to analyse...
Sending titles for classification to Gemini model...
Input is of length: 20.
Wrote 1 lines to Links sheet in the workbook
Trying feed for city: Bangalore...
Found 20 news articles for Bangalore. Now processing to analyse...
Sending titles for classification to Gemini model...
Input is of length: 20.
